In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import gmaps

# Google API Key
from api_keys import g_key

In [2]:
bike_trip_data_df = pd.read_csv("trip_data/2010-capitalbikeshare-tripdata.csv")
station_number = bike_trip_data_df['Start station number']
station_address = bike_trip_data_df['Start station']

bike_trip_location = pd.DataFrame(station_number)
bike_trip_location["Station Address"] = ""
bike_trip_location["Lat"] = ""
bike_trip_location["Lng"] = ""
bike_trip_location["Station Address"]= station_address

#len(bike_trip_location['Start station number'].unique())
#bike_trip_location.head()
bike_trip_location_unique = bike_trip_location.drop_duplicates()

In [3]:
bike_trip_location_unique.head()

,Start station number,Station Address,Lat,Lng
0,31208,M St & New Jersey Ave SE,,
1,31209,1st & N St SE,,
2,31600,5th & K St NW,,
4,31100,19th St & Pennsylvania Ave NW,,
5,31109,7th & T St NW,,


In [4]:
#find a lat/lon to each station number from google
city = "Washington DC"
params = {"key": g_key}

for index, row in bike_trip_location_unique.iterrows():
    


    target_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
 

    address = row['Station Address']
    params['address'] = address,city
    
    station_lat_lng = requests.get(target_url,params = params)
    
    station_lat_lng = station_lat_lng.json()
    try:
        bike_trip_location_unique.loc[index, "Lat"] = station_lat_lng["results"][0]["geometry"]["location"]["lat"]
        bike_trip_location_unique.loc[index, "Lng"] = station_lat_lng["results"][0]["geometry"]["location"]["lng"]
        
    except:
        print("Station Not Found... Skipping....")
        bike_trip_location_unique.loc[index, "Lat"] = None
        bike_trip_location_unique.loc[index, "Lng"] = None 

C:\Users\travi\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [5]:
bike_trip_location_unique.head()

,Start station number,Station Address,Lat,Lng
0,31208,M St & New Jersey Ave SE,38.8764,-77.0041
1,31209,1st & N St SE,38.8746,-77.0059
2,31600,5th & K St NW,38.9015,-77.0189
4,31100,19th St & Pennsylvania Ave NW,38.9002,-77.0435
5,31109,7th & T St NW,38.9156,-77.0219


In [6]:
locations = bike_trip_location_unique.dropna()
locations.head()

,Start station number,Station Address,Lat,Lng
0,31208,M St & New Jersey Ave SE,38.8764,-77.0041
1,31209,1st & N St SE,38.8746,-77.0059
2,31600,5th & K St NW,38.9015,-77.0189
4,31100,19th St & Pennsylvania Ave NW,38.9002,-77.0435
5,31109,7th & T St NW,38.9156,-77.0219


In [7]:
#plot locations to google map
gmaps.configure(g_key)


# Store latitude and longitude in locations
map_locations = locations[["Lat", "Lng"]]

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(map_locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…